# Looking for the objects in the catalogs

We will try to find the onjects from the lst in the catalogs TeV, GeV and Simbad.

- PSR B1259-63
- LS 5039
- LS   I +61  132
- HESS J0632+057
- PSR J2032+4127
- 1FGL J1018.6-5856
- HESS J0536-675


In [1]:
import pandas as pd
import re
import sys

In [2]:
regular_expressions = ['.*B12.*63.*',
                       '.*LS.*5039.*',
                       '.*LS.*I.*61.*303.*',
                       '.*J0632.*057.*',
                       '.*J2032.*.*4127.*',
                       '.*1FGL.*J1018.*5856.*',
                       '.*J0536.*67.*',
                    ]

This cell loads data created in simbad.ipynb

In [3]:
tevsimbad = pd.read_csv("data/tevsimbad.txt", sep='\t', index_col=0)
gevsimbad = pd.read_csv("data/gevsimbad.txt", sep='\t', index_col=0)
gevtev = pd.read_csv("data/gevtev.txt", sep='\t', index_col=0)

gevtevsimbad = pd.read_csv("data/gevtevsimbad.txt", sep='\t', index_col=0)
pd_gev = pd.read_csv("data/gev.txt", sep='\t', index_col=0)
pd_tev = pd.read_csv("data/tev.txt", sep='\t', index_col=0)
pd_simbad = pd.read_csv("data/simbad.txt", sep='\t', index_col=0)
pd_gevtev_simbadclasses = pd.read_csv("data/gevtev_simbadclasses.txt", sep='\t', index_col=0)
pd_gevtevxmm_simbadclasses = pd.read_csv('data/gevtevxmm_s.txt',sep = '\t', index_col=0)

The following cell will be useful to determine a string name of a catalog where an object is found. It is not the cleverest way to do it, as if number of elements is changed(another epsilon is used), this dictionary should be corrected manually.

In [4]:
len_catalogs = {len(tevsimbad) : 'tevsimbad',
            len(gevsimbad) :'gevsimbad',
            len(gevtev) :'gevtev',
            len(gevtevsimbad) :'gevtevsimbad',
            len(pd_gev) : 'gev',
            len(pd_tev) : 'tev',
            len(pd_simbad) : 'simbad',
            len(pd_gevtev_simbadclasses) : 'gevtev_simbadclasses',
            len(pd_gevtevxmm_simbadclasses) : 'pd_gevtevxmm_simbadclasses'
               }

In [5]:
name_to_catalog = {'tevsimbad' : tevsimbad,
            'gevsimbad' : gevsimbad,
            'gevtev' : gevtev,
            'gevtevsimbad' : gevtevsimbad,
            'gev' : pd_gev,
            'tev' : pd_tev,
            'simbad' : pd_simbad,
            'gevtev_simbadclasses' : pd_gevtev_simbadclasses,
            'pd_gevtevxmm_simbadclasses' : pd_gevtevxmm_simbadclasses}

In [6]:
gev_columns = ['gev_2FGL_Name', 'gev_1FGL_Name', 
                 'gev_1FHL_Name', 'gev_2FGL_Name', 
                 'gev_ASSOC1', 'gev_ASSOC2',
                'gev_ASSOC_GAM1', 'gev_ASSOC_GAM2',
                'gev_ASSOC_GAM3', 'gev_ASSOC_TEV', 'gev_CLASS1',
                'gev_GLAT', 'gev_GLON']

tev_columns = ['tev_common_name', 'tev_gamma_names', 
                'tev_fermi_names', 'tev_other_names', 
                'tev_classes',
                'tev_glat', 'tev_glon']

s_columns = ['s_class', 's_spectra', 's_glat', 's_glon',
            's_MAIN_ID']

xmm_columns = ['xmm_IAUNAME']

s_tev_columns = tev_columns + s_columns
s_gev_columns = gev_columns + s_columns
tev_gev_columns = gev_columns + tev_columns
tev_gev_columns_s = gev_columns + tev_columns + ['s_class', 's_simbad', 's_MAIN_ID']
gevtevs_columns = gev_columns + tev_columns + s_columns
tev_gev_xmm_columns_s = gev_columns + tev_columns + xmm_columns + ['s_class', 's_simbad', 's_MAIN_ID']

In [7]:
name_to_column = {'tevsimbad' : s_tev_columns,
            'gevsimbad' : s_gev_columns,
            'gevtev' : tev_gev_columns,
            'gevtevsimbad' : gevtevs_columns,
            'gev' : gev_columns,
            'tev' : tev_columns,
            'simbad' : s_columns,
            'gevtev_simbadclasses' : tev_gev_columns_s,
            'pd_gevtevxmm_simbadclasses' : tev_gev_xmm_columns_s}

In [8]:
def find_in_catalogs(string_to_find, catalog):
    """Looks for an object containing in its name string_to_find in the catalog
    
    Inputs:
    string_to_find - string containing a part og a name that we are looking for
    catalog - in which catalog to look for (pandas DataFrame)
    """
    for c in catalog.columns:
        for obj in catalog[c]:
            if (str(obj)).find(string_to_find) != -1:
                print(len_catalogs[len(catalog)] + ' :')
                print('\t' + str(c))
                print('\t\t' + (str(obj)))
            


In [9]:
def old_find_number_in_catalogs(string_to_find, catalog, printRes = False):
    #Based on string find
    """Looks for an object containing in its name string_to_find in the catalog,
    Finds its number in the corresponging catalog
    
    Inputs:
    string_to_find - string containing a part og a name that we are looking for
    catalog - in which catalog to look for (pandas DataFrame)
    printRes - True if you want to see the search
    
    Returns:
    success - if found is equal to 1, if not - 0
    catalog_name - name of the catalog where string was found(string)
    number_obj - number of a corresponding line
    
    """
    success = 0
    catalog_name = '' 
    number_obj = -1
    for c in catalog.columns:
        for number_obj in range(len(catalog[c])):
            obj = catalog[c][number_obj]
            if (str(obj)).find(string_to_find) != -1:
                success = 1
                
                catalog_name = len_catalogs[len(catalog)]
                
                if printRes:
                    print(len_catalogs[len(catalog)] + ' :')
                    print('\t' + str(c))
                    print('\t\t' + (str(obj)))
                    print('\t\t\t' + str(number_obj))
                return success, catalog_name, number_obj
                
    return success, catalog_name, number_obj

In [10]:
def find_number_in_catalogs(regular_expression, catalog, printRes = False):
    #based on regular expressions
    """Looks for an object containing in its name string_to_find in the catalog,
    Finds its number in the corresponging catalog
    
    Inputs:
    string_to_find - string containing a part og a name that we are looking for
    catalog - in which catalog to look for (pandas DataFrame)
    printRes - True if you want to see the search
    
    Returns:
    success - if found is equal to 1, if not - 0
    catalog_name - name of the catalog where string was found(string)
    number_obj - number of a corresponding line
    
    """
    success = 0
    catalog_name = '' 
    number_obj = -1
    for c in catalog.columns:
        for number_obj in range(len(catalog[c])):
            obj = catalog[c][number_obj]
            if (re.search(regular_expression, str(obj)) is not None):
                success = 1
                
                catalog_name = len_catalogs[len(catalog)]
                
                if printRes:
                    print(len_catalogs[len(catalog)] + ' :')
                    print('\t' + str(c))
                    print('\t\t' + (str(obj)))
                    print('\t\t\t' + str(number_obj))
                return success, catalog_name, number_obj
                
    return success, catalog_name, number_obj

In [11]:
catalogs = [gevtevsimbad,
            tevsimbad,
            gevsimbad,
            gevtev,
            pd_gevtev_simbadclasses,
            pd_gevtevxmm_simbadclasses
           ]

In [12]:
separate_catalogs = [
    pd_gev,
    pd_tev,
    pd_simbad,
]

Strings we want to find in the catalogs:

In [13]:
strings_to_find = [
    'PSR B1259-63',
    'LS 5039',
    'LS   I +61  303',
    'HESS J0632+057',
    'LAT PSR J2032+4127',
    '1FGL J1018.6-5856',
    'J0536',
    
]

In [14]:
resultats_of_search = {}
#this dictionary remembers catalog name and corresponding number of line where an interesting objects is found
for i in len_catalogs.values():
    resultats_of_search.update({i : []})

In [15]:
for regular_expression in regular_expressions:
    print('Looking for : ' + regular_expression)
    for j in catalogs:
        success, catalog_name, number_obj = find_number_in_catalogs(regular_expression, j)
        if success:
            resultats_of_search[catalog_name].append(number_obj)
print('Done')

Looking for : .*B12.*63.*
Looking for : .*LS.*5039.*
Looking for : .*LS.*I.*61.*303.*
Looking for : .*J0632.*057.*
Looking for : .*J2032.*.*4127.*
Looking for : .*1FGL.*J1018.*5856.*
Looking for : .*J0536.*67.*
Done


In [16]:
print(resultats_of_search)

{'tevsimbad': [74, 114, 59, 60, 91], 'gevsimbad': [146, 91, 66, 122], 'gevtev_simbadclasses': [90, 90, 11, 11, 103, 103, 40, 40], 'gevtevsimbad': [60, 9, 7, 38], 'gev': [], 'tev': [], 'simbad': [], 'pd_gevtevxmm_simbadclasses': [4158, 190, 4667, 1012]}


Looking in separate catalogs:

In [17]:
#for regular_expression in regular_expressions:
#    print('Looking for : ' + regular_expression)
#    for j in separate_catalogs:
#        success, catalog_name, number_obj = find_number_in_catalogs(regular_expression, j)
#        if success:
#            resultats_of_search[catalog_name].append(number_obj)
#print('Done')

In [18]:
print(resultats_of_search)

{'tevsimbad': [74, 114, 59, 60, 91], 'gevsimbad': [146, 91, 66, 122], 'gevtev_simbadclasses': [90, 90, 11, 11, 103, 103, 40, 40], 'gevtevsimbad': [60, 9, 7, 38], 'gev': [], 'tev': [], 'simbad': [], 'pd_gevtevxmm_simbadclasses': [4158, 190, 4667, 1012]}


In [19]:
for i in resultats_of_search.keys():
    print(i + ' :')
    display(name_to_catalog[i][name_to_column[i]].loc[resultats_of_search[i]])

tevsimbad :


,tev_common_name,tev_gamma_names,tev_fermi_names,tev_other_names,tev_classes,tev_glat,tev_glon,s_class,s_spectra,s_glat,s_glon,s_MAIN_ID
74,b'PSR B1259-63',b'HESS J1302-638',b'',b'SS 2883',b'bin',-0.991597,304.183594,B0Ve,False,-0.991584,304.183582,b'CPD-63 2495'
114,b'LS 5039',b'HESS J1826-148',b'',b'RX J1826.2-1450',b'bin',-1.289216,16.881586,B0Ve,False,-1.289212,16.881587,b'V* V479 Sct'
59,b'LS I +61 303',b'VER J0240+612',b'',b'',b'bin',1.086135,135.675278,B0Ve,False,1.086129,135.675281,b'LS I +61 303'
60,b'HESS J0632+057',"b'HESS J0632+057,VER J0633+057'",b'',b'HD 259440',b'bin',-1.439528,205.666534,B0Ve,False,-1.439523,205.666547,b'HD 259440'
91,b'HESS J1018-589 A',b'HESS J1018-589 A',b'1FGL J1018.6-5856',"b'SNR G284.3-01.8,MSH 10-53'",b'bin',-1.779517,284.314087,B0Ve,True,-1.678443,284.361358,b'2FGL J1019.0-5856'


gevsimbad :


,gev_2FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,gev_ASSOC_TEV,gev_CLASS1,gev_GLAT,gev_GLON,s_class,s_spectra,s_glat,s_glon,s_MAIN_ID
146,b'2FGL J1826.3-1450 ',b'1FGL J1826.2-1450 ',b'1FHL J1826.2-1450 ',b'2FGL J1826.3-1450 ',b'LS 5039 ',b' ',b' ',b' ',b' ',b'LS 5039 ',b'HMB ',-1.293795,16.885555,B0Ve,False,-1.289212,16.881587,b'V* V479 Sct'
91,b'2FGL J0240.5+6113 ',b'1FGL J0240.5+6113 ',b'1FHL J0240.2+6113 ',b'2FGL J0240.5+6113 ',b'LS I+61 303 ',b' ',b'1AGL J0242+6111',b' ',b'EGR J0240+6112 ',b'LS I +61 303 ',b'HMB ',1.086277,135.678848,B0Ve,False,1.086129,135.675281,b'LS I +61 303'
66,b'2FGL J2032.2+4126 ',b'1FGL J2032.2+4127 ',b'1FHL J2032.1+4125 ',b'2FGL J2032.2+4126 ',b'LAT PSR J2032+4127 ',b' ',b'1AGL J2032+4102',b' ',b' ',b'TeV J2032+4130 ',b'PSR ',1.018827,80.218597,B0Ve,False,1.027891,80.223831,b'[MT91] 213'
122,b'2FGL J1019.0-5856 ',b'1FGL J1018.6-5856 ',b'1FHL J1018.9-5855 ',b'2FGL J1019.0-5856 ',b'1FGL J1018.6-5856 ',b' ',b' ',b' ',b' ',b'HESS J1018-589 ',b'HMB ',-1.690106,284.351227,B0Ve,True,-1.678443,284.361358,b'2FGL J1019.0-5856'


gevtev_simbadclasses :


...

,tev_common_name,tev_gamma_names,tev_fermi_names,tev_other_names,tev_classes,tev_glat,tev_glon


simbad :


,s_class,s_spectra,s_glat,s_glon,s_MAIN_ID


pd_gevtevxmm_simbadclasses :


,gev_2FGL_Name,gev_1FGL_Name,gev_1FHL_Name,gev_2FGL_Name,gev_ASSOC1,gev_ASSOC2,gev_ASSOC_GAM1,gev_ASSOC_GAM2,gev_ASSOC_GAM3,gev_ASSOC_TEV,...,tev_gamma_names,tev_fermi_names,tev_other_names,tev_classes,tev_glat,tev_glon,xmm_IAUNAME,s_class,s_simbad,s_MAIN_ID
4158,b'2FGL J1826.3-1450 ',b'1FGL J1826.2-1450 ',b'1FHL J1826.2-1450 ',b'2FGL J1826.3-1450 ',b'LS 5039 ',b' ',b' ',b' ',b' ',b'LS 5039 ',...,b'HESS J1826-148',b'',b'RX J1826.2-1450',b'bin',-1.289216,16.881586,b'3XMM J182549.2-145024',['B0Ve'],1,"[""b'V* V479 Sct'""]"
190,b'2FGL J0240.5+6113 ',b'1FGL J0240.5+6113 ',b'1FHL J0240.2+6113 ',b'2FGL J0240.5+6113 ',b'LS I+61 303 ',b' ',b'1AGL J0242+6111',b' ',b'EGR J0240+6112 ',b'LS I +61 303 ',...,b'VER J0240+612',b'',b'',b'bin',1.086135,135.675278,b'3XMM J024005.3+611247',['B0Ve'],1,"[""b'LS I +61 303'"" ""b'LS I +61 303'""]"
4667,b'2FGL J2032.2+4126 ',b'1FGL J2032.2+4127 ',b'1FHL J2032.1+4125 ',b'2FGL J2032.2+4126 ',b'LAT PSR J2032+4127 ',b' ',b'1AGL J2032+4102',b' ',b' ',b'TeV J2032+4130 ',...,b'',b'',"b'TeV J2032+4130,TeV 2032+413,0FGL J2032.2+4122'",b'unid',1.111117,80.299477,b'3XMM J203145.8+412815',['B0Ve'],1,"[""b'[MT91] 213'"" ""b'Schulte 21'""]"
1012,b'2FGL J1019.0-5856 ',b'1FGL J1018.6-5856 ',b'1FHL J1018.9-5855 ',b'2FGL J1019.0-5856 ',b'1FGL J1018.6-5856 ',b' ',b' ',b' ',b' ',b'HESS J1018-589 ',...,b'HESS J1018-589 A',b'1FGL J1018.6-5856',"b'SNR G284.3-01.8,MSH 10-53'",b'bin',-1.779517,284.314087,b'3XMM J101826.8-585838',['B0Ve'],2,"[""b'2FGL J1019.0-5856'""]"


The following cell saves information about objects found.

In [20]:
#gevtevsimbad[gevtevs_columns].to_csv('resultat_gevtevsimbad.txt',sep = '\t')
#gevtev[tev_gev_columns].to_csv('resultat_gevtev.txt',sep = '\t')
#gevsimbad[s_gev_columns].to_csv('resultat_sgev.txt',sep = '\t')
#tevsimbad[s_tev_columns].to_csv('resultat_stev.txt',sep = '\t')

In [21]:
#for i in resultats_of_search.keys():
#    (name_to_catalog[i].loc[resultats_of_search[i]]).to_csv('resultats/resultat_' + i +'.txt',sep = '\t')

The following cell adds to bases used in the beginning a column that indicates the objects choosen in this ipynb.

In [22]:
for i in resultats_of_search.keys():
    print(i)
    if i in ['gev','tev','simbad']:
        continue
    catalog = name_to_catalog[i].copy()
    catalog.is_copy = False
    catalog['marked'] = 0
    (catalog.loc[resultats_of_search[i], 'marked'])= 1
    (catalog).to_csv('markeddata/' + i +'.txt',sep = '\t')


tevsimbad
gevsimbad
gevtev_simbadclasses


C:\Users\Admin\Anaconda3\lib\site-packages\pandas\core\generic.py:4388: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  object.__getattribute__(self, name)
C:\Users\Admin\Anaconda3\lib\site-packages\pandas\core\generic.py:4389: FutureWarning: Attribute 'is_copy' is deprecated and will be removed in a future version.
  return object.__setattr__(self, name, value)


gevtevsimbad
gev
tev
simbad
pd_gevtevxmm_simbadclasses
